# Algorithm

### Import modules

In [5]:
import numpy as np
import random 
import time
import csv
from datetime import datetime
from datetime import timedelta
import pandas as pd
from statistics import mean
import logging
import uuid

In [6]:
logging.basicConfig(filename="log_pacing.log", level=logging.INFO)

### Creation of the pacing Class

In [15]:
class Algo:
    def __init__(self, daily_budget, nb_hours_day):
        """Class constructor"""
        # Fixed attributes
        self.daily_budget = daily_budget
        self.target = daily_budget/(nb_hours_day*3600)
        # Impossible day to initialize the setup
        self.day = 0
    
    def buying_decision(self, ts, price):
        """From a BR, decide whether to buy or not
        
        Arguments:
        :ts: timestamp of the BR
        :price: price of the BR
        """
        # TS de la br
        day = datetime.fromtimestamp(ts).day
        month = datetime.fromtimestamp(ts).month
        year = datetime.fromtimestamp(ts).year
        # If we begin a new day, we reset variables
        if self.day != day:
            logging.info("-\n CHANGING DAY \n")
            self.remaining_budget = self.daily_budget
            self.BT = [self.target]
            self.speed = pd.DataFrame({'V':0}, index=[datetime(year,month,day,6,0,0,0)])
            self.engaged_budget = 0
            self.spent_budget = 0
        self.day = day
        # On calcule le temps restant avant la fin de la journée
        end_day = datetime(year,month,day,19,40,0,0)
        remaining_time =  datetime.timestamp(end_day) - ts
        # calcul de bt
        created_time = self.speed.index[-1] - timedelta(minutes=30)
        self.remaining_budget = self.daily_budget - (self.engaged_budget + self.spent_budget)
        try:
            bt = self.remaining_budget * ((1+100*self.speed.V[self.speed.index > created_time].mean()) / remaining_time) 
        except ZeroDivisionError:
            bt = 1
        if bt < 0:
            bt = 1
        self.BT.append(bt)
        # Calcul de vt
        vt = self.BT[-1] - self.BT[-2] 
        self.speed = self.speed.append(pd.DataFrame({'V':vt}, index=[datetime.fromtimestamp(ts)]))
        if (bt >= self.target or bt<=0) and (self.remaining_budget - price) >= 0:
            buying = True
            self.engaged_budget += price
        else:
            buying = False

        return (buying, bt, vt)     

    def send_pending_notifications(self,current_ts = None):
        """ Send notifications 
        
        :param current_ts: if None: will send all notifications, else send before current_ts
        :return:
        """
        while len(pending_notifications) > 0 and (pending_notifications[0]['timestamp'] <= current_ts if current_ts else True):
            ev = pending_notifications.pop(0)
            logging.info(f"{ev['timestamp']} -> Sending {ev['status']} notif with ID {ev['id']}")
            if ev['status'] == 'win':
                self.engaged_budget -= ev['br_price']
                self.spent_budget += ev['br_price']
            else:
                self.engaged_budget -= ev['br_price']

In [16]:
data = pd.read_csv('09-07-2020_07-08-2020.csv', index_col="timestamp_string", parse_dates=True)
data.index.names = ['Date']
data.head()

,ID,timestamp,nb_imp,price,win,seconds_notif
Date,,,,,,
2020-07-09 06:00:00,1,1.594267e+09,1,1,True,287
2020-07-09 06:00:00,2,1.594267e+09,3,3,True,893
2020-07-09 06:00:00,3,1.594267e+09,8,8,True,708
2020-07-09 06:00:00,4,1.594267e+09,3,3,True,729
2020-07-09 06:00:03,5,1.594267e+09,4,4,True,707


In [17]:
pacing = Algo(daily_budget=3000, nb_hours_day=14)

In [18]:
buyings = list()
remaining = list()
bt = list()
target = list()
vt = list()
spent = list()
engaged = list()
pending_notifications = list()
day = 9
for current_ts, row in data.iterrows():
    # Send current notifications
    pacing.send_pending_notifications(current_ts)
    if current_ts.day != day:
        day = current_ts.day
        engaged[-1] = pacing.engaged_budget
        pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
        remaining[-1] = pacing.remaining_budget
        spent[-1] = pacing.spent_budget
    
    # Receive BR
    decision = pacing.buying_decision(row['timestamp'], row['price'])
    logging.info(f"{current_ts} -> sending BR")
    # Making a decision
    if decision[0]:
        #Buying
        logging.info(f"{current_ts} <- receiving buy")
        next_notif_ts = current_ts + timedelta(seconds=row['seconds_notif'])
        status = "win" if row['win'] else "lose"
        notif_id = uuid.uuid4()
        logging.info(f" | {current_ts} decided {status}, will be notified at {next_notif_ts} with ID {notif_id}")
        pending_notifications.append({"timestamp": next_notif_ts, "status": status, 'br_price': row['price'], 'id': notif_id})
        pending_notifications.sort(key=lambda x: x['timestamp'])
    target.append(pacing.target)
    bt.append(decision[1])
    vt.append(decision[2])
    buyings.append(decision[0])
    remaining.append(pacing.remaining_budget)
    spent.append(pacing.spent_budget)
    engaged.append(pacing.engaged_budget)
# Send remaining notifications
pacing.send_pending_notifications()
# Update last row after sending last notifications
engaged[-1] = pacing.engaged_budget
pacing.remaining_budget = pacing.daily_budget - (pacing.engaged_budget + pacing.spent_budget)
remaining[-1] = pacing.remaining_budget
spent[-1] = pacing.spent_budget
data['target'] = target
data['bt'] = bt
data['vt'] = vt
data['buying'] = buyings
data['remaining_budget'] = remaining
data['engaged_budget'] = engaged
data['spent_budget'] = spent

In [7]:
data.to_csv('pacing_08-07-2020_06-08-2020.csv', index=True)

In [20]:
data.groupby(data.index.day).tail(1)

,ID,timestamp,nb_imp,price,win,seconds_notif,target,bt,vt,buying,remaining_budget,engaged_budget,spent_budget
Date,,,,,,,,,,,,,
2020-07-09 19:59:49,1544,1.594318e+09,2,2,True,578,0.059524,-0.0,0.0,False,0,0,3000
2020-07-10 19:59:57,3162,1.594404e+09,1,1,True,440,0.059524,-0.0,0.0,False,0,0,3000
2020-07-11 19:59:50,4707,1.594490e+09,4,4,True,770,0.059524,-0.0,0.0,False,0,0,3000
2020-07-12 19:59:26,6343,1.594577e+09,1,1,True,854,0.059524,-0.0,0.0,False,0,0,3000
2020-07-13 19:59:59,7954,1.594663e+09,4,4,False,552,0.059524,1.0,0.0,False,1,0,2999
2020-07-14 19:59:54,9716,1.594750e+09,10,10,True,312,0.059524,-0.0,0.0,False,4,0,2996
2020-07-15 19:59:53,11363,1.594836e+09,11,11,True,414,0.059524,-0.0,0.0,False,0,0,3000
2020-07-16 19:59:48,12723,1.594922e+09,0,0,True,680,0.059524,-0.0,0.0,True,0,0,3000
2020-07-17 19:57:49,14394,1.595009e+09,3,3,True,466,0.059524,-0.0,0.0,False,0,0,3000
